In [3]:
# Download checkpoint to exp/bigvgan
GENERATOR_URL = ""
DISCRIMINATOR_URL = ""
CONFIG_URL = ""
DATASET_ZIP = ""
TRAIN_LIST_URL = ""
VAL_LIST_URL = ""
UNSEEN_LIST_URL = ""
BATCH_SIZE = 2
data_zip_path = "data.zip"
checkpoint_path = "exp/bigvgan"
config_path = "configs/config.json"
input_wavs_dir = "acoustic_rec_data"
bigvgan_root = "/root/BigVGAN"

def download_file(url, dest):
    print(f"Downloading {url} to {dest}")
    with urllib.request.urlopen(url) as r, open(dest, 'wb') as out_file:
        total_size = int(r.info().get('Content-Length', 0))
        block_size = 1024
        with tqdm(total=total_size, unit='B', unit_scale=True) as pbar:
            while True:
                data = r.read(block_size)
                if not data:
                    break
                out_file.write(data)
                pbar.update(len(data))

import os
from pathlib import Path
import yaml
import json
bigvgan_root = Path(bigvgan_root)
os.makedirs(bigvgan_root/checkpoint_path, exist_ok=True)
os.makedirs(bigvgan_root/"configs", exist_ok=True)

# checkpoint downloads
g_path = bigvgan_root / "g_00000000"
do_path = bigvgan_root / "do_00000000"
if not os.path.exists(g_path):
    download_file(GENERATOR_URL, g_path)
if not os.path.exists(do_path):
download_file(DISCRIMINATOR_URL, do_path)

# dataset downloads
if not os.path.exists(data_zip_path):
    download_file(DATASET_ZIP, data_zip_path)
if not os.path.exists(bigvgan_root / "train_list.txt"):
    download_file(TRAIN_LIST_URL, bigvgan_root / "train_list.txt")
if not os.path.exists(bigvgan_root / "val_list.txt"):
    download_file(VAL_LIST_URL, bigvgan_root / "val_list.txt")
if not os.path.exists(bigvgan_root / "unseen_list.txt"):
    download_file(UNSEEN_LIST_URL, bigvgan_root / "unseen_list.txt")
if not os.path.exists(input_wavs_dir):
    with zipfile.ZipFile(dataset_zip_path, 'r') as f:
        files_count = len(f.namelist())
        with tqdm(
            total=files_count, desc="Extracting files", unit="file") as pbar:
            for info in f.namelist():
                unzip_file_path = Path(DATASET_TARGET_DIR)
                if (unzip_file_path/info).exists():
                    continue
                f.extract(info, unzip_file_path)
                pbar.update(1)
with open(config_path,'r') as f:
    config = json.load(f)
    config['batch_size'] = BATCH_SIZE

\root\BigVGAN\exp\bigvgan


In [ ]:
!accelerate launch train.py --config configs/{config_path} \
    --input_wavs_dir {bigvgan_root/input_wavs_dir} \
    --input_training_file {bigvgan_root/"train_list.txt"} \
    --input_validation_file {bigvgan_root/"val_list.txt"} \ 
    --list_input_unseen_wavs_dir {bigvgan_root/"unseen"} \ 
    --list_input_unseen_validation_file {bigvgan_root/"unseen_list.txt"} \
    --validation_interval 8000 \
    --eval_subsample 100